In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# 2D power spectrum estimation

This notebook is used to estimate the cylindrical power spectrum from one or more data cubes.

First, we import required modules and define a function to calculate the PS (see *PS estimation with ps_eor* notebook for detailed info).

In [2]:
import os
import configparser
import numpy as np
from ps_eor import psutil, datacube, pspec
import matplotlib.pyplot as plt
import astropy.units as u
import matplotlib.pyplot as plt

In [3]:
def calc_all_ps(impath, psfpath, savepath, team, freqs, min_b, max_b, fov, int_time, tot_time, nbins_per, nbins_par, binmin, binmax, replace_freq=True):
    os.makedirs(savepath, exist_ok=True)
    for i in range(len(freqs)-1):
        print('----- Bin %s -----'%i)
        # read parameters
        if replace_freq:
            image = impath+'_%sMHz-%sMHz.fits'%(freqs[i],freqs[i+1])
        else:
            if i != len(freqs)-2:
                image = impath+'_%sMHz-%sMHz.fits'%(str(i*150),str((i+1)*150))
            else:
                image = impath+'_%sMHz-%sMHz.fits'%(str(i*150),str((i+1)*150+1))
        psf = psfpath+'_%sMHz-%sMHz.fits'%(freqs[i],freqs[i+1])
        data_cube = datacube.CartDataCube.load_from_fits_image_and_psf([image],
                [psf], min_b, max_b, fov,
                        use_wscnormf=False, int_time=int_time, total_time=tot_time,
                        window_function=datacube.WindowFunction(('tukey', 0.2)))
        
        # define el
        z = psutil.freq_to_z((freqs[i]+freqs[i+1])/2*1e6) # give in Hz
        el = psutil.k_to_l(np.linspace(binmin, binmax, nbins_per), z)

        #this is the number of bins we would get using the bins from data_cube.freqs
        calc_k_par = len(psutil.delay_to_k(psutil.get_delay(data_cube.freqs), z))-1 
        par_binmax = (binmax-binmin)/(nbins_par-1)*calc_k_par #get more than binmax for better calculation
        mfreq = (freqs[i]+freqs[i+1]) * 1e6 / 2  # in Hz
        delay = psutil.k_to_delay(np.linspace(binmin, par_binmax, calc_k_par), z)
        fwidth = 2*calc_k_par / ((delay[1]-delay[0])*(2*calc_k_par+1))
        freqbins = np.linspace(mfreq - fwidth/2, mfreq + fwidth/2, calc_k_par*2+1)

        # estimate ps
        eor = pspec.EorBin(1, freqbins, freqbins, M=len(freqbins))
        ps_conf = pspec.PowerSpectraConfig(el, window_fct='blackmanharris')
        pb = datacube.SkaLowPrimaryBeam()
        ps_gen = pspec.PowerSpectraCart(eor, ps_conf, pb)
        ps2d = ps_gen.get_ps2d(data_cube)

        np.savetxt(f'{savepath}{team}_{freqs[i]}MHz-{freqs[i+1]}MHz.data', ps2d.data[:nbins_par, :nbins_per], fmt='%.8g')
        np.savetxt(f'{savepath}{team}_{freqs[i]}MHz-{freqs[i+1]}MHz_errors.data', ps2d.err[:nbins_par, :nbins_per], fmt='%.8g')

Next, we will read the parameters from the configuration file (*ps_config.ini*)

In [4]:
config_object = configparser.ConfigParser()
with open("ps_config.ini","r") as file_object:
    config_object.read_file(file_object)
    # read paths
    impath = config_object.get("path","image")
    psfpath = config_object.get("path","psf")
    savepath = config_object.get("path","save")
    team = config_object.get("path","team")
    # read params
    min_b = float(config_object.get("params","min_b"))
    max_b = float(config_object.get("params","max_b"))
    fov = np.deg2rad(float(config_object.get("params","fov")))
    int_time = float(config_object.get("params","int_time"))
    tot_time = float(config_object.get("params","tot_time"))
    freqs = np.linspace(float(config_object.get("params","min_freq")), float(config_object.get("params","max_freq")), int(config_object.get("params","nbins_freq"))+1)
    nbins_per = int(config_object.get("params","nbins_per"))
    nbins_par = int(config_object.get("params","nbins_par"))
    binmin = float(config_object.get("params","binmin"))
    binmax = float(config_object.get("params","binmax"))
    # read cosmological parameters
    H0 = float(config_object.get("cosmo","H0"))
    Om0 = float(config_object.get("cosmo","Om0"))

We set the cosmological parameters. **ps_eor** uses the *Planck15* best fit, so it must be changed.

In [5]:
from astropy.cosmology import FlatLambdaCDM
new_cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

psutil.set_cosmology(new_cosmo)

Finally, we run the function. This will create a set of files with the PS at the required frequency bins, with the SDC3 submission format; another set with the errors at the same format.

In [6]:
calc_all_ps(impath, psfpath, savepath, team, freqs, min_b, max_b, fov, int_time, tot_time, nbins_per, nbins_par, binmin, binmax, replace_freq=True)

 Progress: 1 / 1 (Total: 0.00 s) 
 Progress: 1 / 1 (Total: 0.00 s) 
Filtering 146801 modes (72.49 %)
 Progress: 1 / 1 (Total: 0.00 s) 
 Progress: 1 / 1 (Total: 0.00 s) 
Filtering 146159 modes (72.18 %)
 Progress: 1 / 1 (Total: 0.00 s) 
 Progress: 1 / 1 (Total: 0.00 s) 
Filtering 145281 modes (71.74 %)
 Progress: 1 / 1 (Total: 0.00 s) 
 Progress: 1 / 1 (Total: 0.00 s) 
Filtering 146617 modes (72.40 %)
 Progress: 1 / 1 (Total: 0.00 s) 
 Progress: 1 / 1 (Total: 0.00 s) 
Filtering 145045 modes (71.63 %)
 Progress: 1 / 1 (Total: 0.00 s) 
 Progress: 1 / 1 (Total: 0.00 s) 
Filtering 144923 modes (71.57 %)
